<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 13th exercise: <font color="#C70039">First Reinforcement Learning Q-Table learning</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>

* **Student: Tzu-Lun Yeh**
* **Matriculation Number: 11496498**
* **Date: 10.12.2025**

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. play with all hyperparameters including the actions, states, rewards table.
5. add and implement an ϵ-greedy strategy 
---------------------------------

In [1]:
# Imports 
import numpy as np

### Create the possible states

In [2]:
location_to_state = {
    'L1' : 0,
    'L2' : 1,
    'L3' : 2,
    'L4' : 3,
    'L5' : 4,
    'L6' : 5,
    'L7' : 6,
    'L8' : 7,
    'L9' : 8
}

state_to_location = dict((state,location) for location, state in location_to_state.items())

### Create the actions & rewards

In [3]:
actions = [0,1,2,3,4,5,6,7,8]

In [4]:
rewards = np.array([[0,1,0,0,0,0,0,0,0],
                   [1,0,1,0,1,0,0,0,0],
                   [0,1,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,1,0,0],
                   [0,1,0,0,0,0,0,1,0],
                   [0,0,1,0,0,0,0,0,0],
                   [0,0,0,1,0,0,0,1,0],
                   [0,0,0,0,1,0,1,0,1],
                   [0,0,0,0,0,0,0,1,0]])

### Def remaining hyperparameters

In [5]:
# initialize the parameters
gamma = 0.99  # discount factor
alpha = 0.9   # learning rate

# ε-greedy parameters
epsilon = 1.0              # initial exploration rate (100% exploration at start)
max_epsilon = 1.0          # maximum exploration rate
min_epsilon = 0.01         # minimum exploration rate (always keep 1% exploration)
epsilon_decay = 0.01      # exploration decay rate (based on Ex12 best practice)

### Define agent and its attributes

### ε-greedy Strategy Implementation

The ε-greedy strategy balances **exploration** (trying new actions) and **exploitation** (using learned knowledge).

**How it works:**
- With probability ε: randomly explore (choose random action)
- With probability (1-ε): exploit knowledge (choose action with highest Q-value)
- ε starts at 1.0 (100% exploration) and decays exponentially to 0.01 (1% exploration)

**Benefits:**
- Early training: High exploration helps discover environment
- Late training: High exploitation uses learned Q-table effectively
- Always maintains small exploration rate to avoid local optima

**Parameters (based on Ex12 experiments):**
- `epsilon_decay = 0.001`: Slower decay performs better than 0.01
- `min_epsilon = 0.01`: Always keep 1% exploration chance

**Key Optimization:**
When multiple actions have the same Q-value (especially during early training), we randomly select among them instead of always picking the first one. This prevents directional bias and improves learning efficiency.

In [6]:
class QAgent():
    # initialize everything
    def __init__(self, alpha, gamma, location_to_state, actions, rewards, state_to_location,
                 epsilon, max_epsilon, min_epsilon, epsilon_decay):
        self.gamma = gamma
        self.alpha = alpha
        
        self.location_to_state = location_to_state
        self.actions = actions
        self.rewards = rewards
        self.state_to_location = state_to_location
        
        # ε-greedy parameters
        self.epsilon = epsilon
        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.epsilon_decay = epsilon_decay
        
        # remember, the Q-value table is of size all actions x all states
        M = len(location_to_state)
        self.Q = np.zeros((M,M), dtype = None, order = 'C')
        
    # now, implement the training method for the agent
    def training(self, start_location, end_location, iterations):

        rewards_new = np.copy(self.rewards)

        ending_state = self.location_to_state[end_location]
        
        rewards_new[ending_state, ending_state] = 999
        
        # DEBUG
        print(rewards_new)

        # pick random current state
        # iterations = the # of training cycles
        for i in range(iterations):
            current_state = np.random.randint(0,9)
            playable_actions = []

            # iterate thru the rewards matrix to get states
            # that are really reachable from the randomly chosen
            # state and assign only those in a list since they are really playable
            for j in range(9):
                if rewards_new[current_state, j] > 0:
                    playable_actions.append(j)

            # ε-greedy action selection strategy
            if len(playable_actions) != 0:
                # Generate random threshold for exploration vs exploitation decision
                exploration_threshold = np.random.random()

                if exploration_threshold > self.epsilon:
                    # Exploitation: choose action with highest Q-value
                    # Get Q-values for all playable actions from current state
                    q_values = [self.Q[current_state, action] for action in playable_actions]

                    # Optimization: handle ties in Q-values by random selection
                    # This avoids bias when multiple actions have the same max Q-value
                    max_q = np.max(q_values)
                    indices_of_max = [i for i, v in enumerate(q_values) if v == max_q]
                    best_action_index = np.random.choice(indices_of_max)
                    next_state = playable_actions[best_action_index]
                else:
                    # Exploration: randomly select from playable actions
                    next_state = np.random.choice(playable_actions)
    
            # finding the difference in Q, often referred to as temporal difference
            # by means of the Bellman's equation (compare with slides)
            TD = rewards_new[current_state, next_state] + self.gamma * self.Q[next_state, np.argmax(self.Q[next_state,])] - self.Q[current_state, next_state]
            self.Q[current_state, next_state] += self.alpha*TD
            
            # Epsilon decay using exponential decay formula (after each iteration)
            # Formula from Ex12 best practice
            self.epsilon = self.min_epsilon + \
                           (self.max_epsilon - self.min_epsilon) * \
                           np.exp(-self.epsilon_decay * i)
            
            # Optional: print epsilon value every 100 iterations for debugging
            if i % 100 == 0:
                print(f"Iteration {i}: epsilon = {self.epsilon:.4f}")

        route = [start_location]
        next_location = start_location

        # print the optimal route from start to end
        self.get_optimal_route(start_location, end_location, next_location, route, self.Q)

    # compute the optimal route
    def get_optimal_route(self, start_location, end_location, next_location, route, Q):
        while(next_location != end_location):
            starting_state = self.location_to_state[start_location]
            next_state = np.argmax(Q[starting_state,])
            next_location = self.state_to_location[next_state]
            route.append(next_location)
            start_location = next_location
        # DEBUG
        print('Q-table:',Q)
        print("optimal route:", route)

In [7]:
qagent = QAgent(alpha, gamma, location_to_state, actions, rewards, state_to_location,
                epsilon, max_epsilon, min_epsilon, epsilon_decay)
qagent.training('L9', 'L4', 1000)

[[  0   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Iteration 0: epsilon = 1.0000
Iteration 100: epsilon = 0.3742
Iteration 200: epsilon = 0.1440
Iteration 300: epsilon = 0.0593
Iteration 400: epsilon = 0.0281
Iteration 500: epsilon = 0.0167
Iteration 600: epsilon = 0.0125
Iteration 700: epsilon = 0.0109
Iteration 800: epsilon = 0.0103
Iteration 900: epsilon = 0.0101
Q-table: [[0.00000000e+00 5.37579168e+04 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [4.10326738e+00 0.00000000e+00 2.65803219e+00 0.00000000e+00
  5.42999431e+04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 5.37331504e+04 0.000

## Experiment Results

### Setup
Training iterations: 1000, Start: L9, Goal: L4

Hyperparameters: alpha=0.7, gamma=0.99, epsilon_decay=0.001

### Observations

**Epsilon decay**: Successfully decreased from 1.0 to 0.4125, showing smooth transition from exploration to exploitation.

**Optimal route found**: L9 -> L8 -> L7 -> L4 (3 steps, shortest path)

**Q-table values**: Range from 36000 to 45000. Highest value at goal state (45298). States leading to goal have higher Q-values as expected.

**Comparison with pure random**: The epsilon-greedy strategy consistently finds optimal path while pure random approach was unstable.

### Conclusion

The epsilon-greedy implementation works correctly. The agent balances exploration and exploitation effectively and learns the optimal policy to reach the goal from any starting position.

### Comparison: 5000 iterations

Tested with 5000 iterations using same hyperparameters.

**Result**: Epsilon decreased to 0.0174 (nearly full exploitation). Q-values approximately doubled (79000-97000 range). However, the optimal route remains identical (L9 → L8 → L7 → L4).

**Observation**: For this simple 9-state environment, increasing iterations from 1000 to 5000 does not improve route quality. 1000 iterations is sufficient to find the optimal policy.

### Comparison: epsilon_decay = 0.01

Tested with epsilon_decay = 0.01 (faster decay rate) using 1000 iterations.

**Result**: Epsilon reached 0.0101 at iteration 900, nearly converged by iteration 500 (epsilon = 0.0167). Optimal route remains L9 → L8 → L7 → L4.

**Advantages**:
1. Extremely fast convergence: reaches final strategy within 500 iterations
2. Efficient training: suitable for simple environments to quickly find solution
3. Time-saving: 500 iterations sufficient, no need for 5000 iterations

**Trade-off**: Q-table learning is uneven. Main path has high Q-values (52K-55K) but non-main paths have very low values (8-25), indicating insufficient exploration of alternative routes.

### Comparison: alpha = 0.9

Tested with alpha = 0.9 (higher learning rate) while keeping epsilon_decay = 0.01 and 1000 iterations.

**Result**: Optimal route unchanged (L9 → L8 → L7 → L4). Main path Q-values increased to 56K-58K range (5-8% higher than alpha=0.7).

**Observations**:

Higher learning rate leads to faster Q-value accumulation on main path. However, non-main paths show extreme instability with values ranging from 2.6 to 27,201, indicating over-sensitivity to individual experiences.

**Comparison with alpha=0.7**:
- Main path: alpha=0.9 slightly higher and faster convergence
- Non-main paths: alpha=0.7 more stable and reliable
- Overall: alpha=0.7 provides better balance between learning speed and stability

**Conclusion**: For this environment, alpha=0.7 is recommended as it achieves good convergence while maintaining stable Q-values across all state-action pairs.